In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('sparkify_etl').getOrCreate()

22/12/28 14:09:53 WARN Utils: Your hostname, Gunnvants-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.0.184 instead (on interface en0)
22/12/28 14:09:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/28 14:09:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
source_songs = '/Users/gunnvantsaini/Library/CloudStorage/OneDrive-Personal/project_codes/backend/data_engineering/cloud_dw/scripts/song_data/*/*/*/*.json'
source_logs = '/Users/gunnvantsaini/Library/CloudStorage/OneDrive-Personal/project_codes/backend/data_engineering/cloud_dw/scripts/log_data/*/*/*.json'

In [4]:
songs = spark.read.json(source_songs)
logs = spark.read.json(source_logs)

In [5]:
songs.count()

14896

In [6]:
logs.count()

8056

In [7]:
songs.printSchema()

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)



In [8]:
logs.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [9]:
user_table_cols = ['userId', 'firstName', 'lastName', 'gender', 'level']
song_table_cols = ['song_id', 'title', 'artist_id', 'year', 'duration']
artist_table_cols = ['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']

In [10]:
user_table=logs.select(user_table_cols).drop_duplicates()
songs_table=songs.select(song_table_cols).drop_duplicates()
artist_table=songs.select(artist_table_cols).drop_duplicates()

In [11]:
from pyspark.sql import functions as f
from pyspark.sql.functions import udf as udf
from pyspark.sql import types as t
@udf(t.FloatType())
def norm_ts(v):
    return v/1000

In [12]:
logs = logs.withColumn("start_time",f.from_unixtime(norm_ts("ts")))

In [13]:
time_table = logs.\
            withColumn('hour',f.hour('start_time')).\
            withColumn("day",f.dayofmonth("start_time")).\
            withColumn("week",f.dayofweek("start_time")).\
            withColumn("month",f.month("start_time")).\
            withColumn("year",f.month("start_time")).distinct()
                                                   

In [14]:
cols = ['start_time', 'userId', 'level', 'song_id', 'artist_id', 'sessionId', 'location', 'userAgent']
song_plays = songs.\
        join(logs.where("page=='NextSong'"),songs.artist_name==logs.artist,'inner').\
        select(cols).distinct().\
        withColumn("songplay_id",f.monotonically_increasing_id()).\
        withColumn("year",f.year("start_time")).\
        withColumn("month",f.month("start_time")).distinct()

In [15]:
songs_table.write.partitionBy(['year','artist_id']).parquet("songs_table")

In [16]:
artist_table.write.parquet("artist_table")

In [17]:
user_table.write.parquet("user_table")

In [18]:
time_table.write.partitionBy('year','month').parquet("time_table")

In [19]:
song_plays.write.partitionBy('year','month').parquet('songplays_table')

22/12/28 15:25:28 WARN JavaUtils: Attempt to delete using native Unix OS command failed for path = /private/var/folders/7h/01sjg2sx6tl3y72r2klm24c80000gn/T/blockmgr-3b887eea-2b37-42de-962b-c31926e161fd. Falling back to Java IO way
java.io.IOException: Failed to delete: /private/var/folders/7h/01sjg2sx6tl3y72r2klm24c80000gn/T/blockmgr-3b887eea-2b37-42de-962b-c31926e161fd
	at org.apache.spark.network.util.JavaUtils.deleteRecursivelyUsingUnixNative(JavaUtils.java:171)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:110)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:91)
	at org.apache.spark.util.Utils$.deleteRecursively(Utils.scala:1206)
	at org.apache.spark.storage.DiskBlockManager.$anonfun$doStop$1(DiskBlockManager.scala:374)
	at org.apache.spark.storage.DiskBlockManager.$anonfun$doStop$1$adapted(DiskBlockManager.scala:370)
	at scala.collection.IndexedSeqOptimized.foreach(IndexedSeqOptimized.scala:36)
	at scala.collection.Indexe

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 58076)
Traceback (most recent call last):
  File "/Users/gunnvantsaini/miniforge3/envs/spark/lib/python3.8/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/Users/gunnvantsaini/miniforge3/envs/spark/lib/python3.8/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/Users/gunnvantsaini/miniforge3/envs/spark/lib/python3.8/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/Users/gunnvantsaini/miniforge3/envs/spark/lib/python3.8/socketserver.py", line 747, in __init__
    self.handle()
  File "/Users/gunnvantsaini/spark-3.3.1-bin-hadoop3/python/pyspark/accumulators.py", line 281, in handle
    poll(accum_updates)
  File "/Users/gunnvantsaini/spark-3.3.1-bin-hadoop3/python/pyspark/accumulators.py", line 253,